# Assistant-11<br/>Travel Assistant

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as helper

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Load the files

In [2]:
## Files
file_itinerary = helper.upload_file(client,"../data/travel/flight_itinerary.txt")
file_bills = helper.upload_file(client,"../data/travel/airbnb_bills.csv")
#file_trains = helper.upload_file(client,"../data/train_ids.csv")
#file_stations = helper.upload_file(client,"../data/stations.csv")

Added file:  assistant-Ul4pGaRffB4FuyrV6d7xolrs 1
Added file:  assistant-DZqLaXltOgKsYpCedixnfbIO 2


### Define the tools to use in the Assistants

In [4]:
tools_list = [
    {"type":"retrieval"},    
    {"type":"code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "block_calendar",
            "description": "Sends a calendar event.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eventSubject": {
                        "type": "string",
                        "description": "Event subject."
                    },
                    "eventContent": {
                        "type": "string",
                        "description": "Event content."
                    },
                    "startTime": {
                        "type": "string",
                        "description": "Event start time and date."
                    },
                    "endTime": {
                        "type": "string",
                        "description": "Event end time and date."
                    },
                    "isAllDay": {
                        "type": "boolean",
                        "description": "Is the event all day."
                    }
                },
                "required": ["eventSubject", "eventContent","startTime","endTime","isAllDay"]
            }
        }
    }
]

### Define the Assistant

In [5]:
helper.clear_shelves()

travel_bot_assistant = helper.create_assistant(client,
        name="Travel Assistant",
        instructions="You are a helpful, friendly, and professional assistant that can help with travel plans, calculate expenses, and answer questions based on the provided knowledge.",        
        tools=tools_list,
        model=helper.gpt_deployment_name,
        file_ids=[file_itinerary.id,file_bills.id],)

Added assistant:  asst_3XW40clwXffKhhdYDHhut1ER 1


### Define the delegate to handle function calling

In [6]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "block_calendar":
            print("blocking calendar...")
            subject = arguments['eventSubject']
            content = arguments['eventContent']
            start = arguments['startTime']
            end = arguments['endTime']
            is_all_day = arguments['isAllDay']
            helper.send_event(subject,content,start,end,is_all_day)
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Calendar blocked"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process user messages

In [7]:
try:
    helper.generate_response(client,travel_bot_assistant,"Suggest a day-by-day itinerary for the dates of my travel? I like walk around the city I'm visiting and go on day trips?", "123", "Alex",function_calling_delegate)
    helper.generate_response(client,travel_bot_assistant,"Will be 4 friends staying at this Airbnb in Berlin. What's my share of costs+my flight?", "123", "Alex",function_calling_delegate)
    helper.generate_response(client,travel_bot_assistant,"Block my calendar tomorrow from 5-5:30pm", "123", "Alex",function_calling_delegate)
finally:
    helper.cleanup(client)

Creating new thread for Alex with user_id 123
Added thread:  thread_3ZcIVFMHptFOd68MUCQ1ByiV 1
Generated message: Based on your provided flight itinerary, you will be in Frankfurt from September 29, 2024, until October 3, 2024. Here's a suggested day-by-day itinerary that will allow you to explore the city and enjoy day trips around the area:

**September 29, 2024 (Day 1 - Arrival in Frankfurt)**:
- Morning: Arrive at Frankfurt Airport at 6:00 AM.
- Transfer to your hotel and check-in. Take some time to rest and freshen up.
- Afternoon: Take a leisurely walk around the Römerberg, Frankfurt's old central square. Visit the Römer building and the Old St Nicholas Church.
- Evening: Enjoy dinner at one of the traditional restaurants in the Sachsenhausen district.

**September 30, 2024 (Day 2 - Exploring Frankfurt)**:
- Morning: Visit the Städel Museum to see an impressive collection of European art.
- Afternoon: Have lunch at Kleinmarkthalle, a local market with a variety of fresh food and 